In [3]:
from os import walk
import json
import re
from tqdm import tqdm
from utils import data_analysis, pmi, pmi_to_dict, docs_distribution, new_distribution
from Scoring import compute_scores
import time

### Fichier qui créé mes inputs (Proba dist for all, for each doc, and for new ones, and PMI calculations)

Si j'ai des textes, ça doit me ressortir des vecteurs de distributions pour mes documents KB, EB, et pour mes nouveaux et les combinaisons

## Pour chaque recette on compute la KB et la EB

thresholds: </br>

Novelty divergence threshold to know if a term significantly contribute (avg = 0.000923, std = 0.00161) -- set to : 0.0041 </br>
Novelty divergence threshold to know if a term significantly contribute in probability of appearing (avg = 14.640, std = 21.253) -- set to : 57.14 </br>

Novelty newness div threshold (avg = 9.787e-05, std = 0.0006) -- set to : 0.0014 </br>
Novelty newness prob threshold (avg = 0.000601, std = 0.00043) -- set to : 0.0014 </br>
Novelty uniqness dist threshold (avg = 0.341, std = 0.093) -- set to : 0.527 </br>
Novelty uniqness shift threshold (avg = 0.0755, std = 0.0270) -- set to : 0.1295 </br>
Novelty difference local threshold (avg = 0.0104, std = 0.0730) -- set to : 0.1564 </br>
Novelty difference global threshold (avg = 0.1099, std = 0.1539) -- set to : 0.4177 </br>
Novelty surprise new threshold (avg = 0.0016, std = 0.0052) -- set to : 0.0104 </br>
Novelty surprise dist threshold (avg = 0.00048, std = 0.00104) -- set to : 0.00256 </br>

In [5]:
my_path = './Recipe_dataset/RecipeFullDataset/'
my_new_path = './Recipe_dataset/Recipe_with_scores/'
save_path = './Recipe_dataset/Recipe_with_scores/'
filenames1 = set(next(walk(my_path), (None, None, []))[2])
filenames2 = set(next(walk(my_new_path), (None, None, []))[2])

filenames = list(filenames1 - filenames2)
print(len(filenames))

6


In [7]:
filenames

['Marinade_recipe_2.json',
 'Tartar_recipe_4.json',
 'Sesame Noodle_recipe_1.json',
 'Honey Cake_recipe_0.json',
 'Chicken Satay_recipe_3.json',
 'Chile Relleno_recipe_236.json']

In [9]:
start_time = time.time()
for k in range(len(filenames)):
    recette = filenames[k]
    file_path = my_path + recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)

    #### COLLECTING ALL NECESSARY INFO
    KB_recettes, _ = data_analysis(recipe_dict)
    print('Knowledge base size : ', len(KB_recettes))
    if len(KB_recettes) <= 0:
        continue

    ### Transforming KB into distribution
    KB_texts = ' '.join(KB_recettes).split()
    EB_PMI = pmi(KB_texts)
    dict_know_pmi = pmi_to_dict(EB_PMI)
    
    train_recettes, train_indexes  = data_analysis(recipe_dict, ref=False, col_name='Train_Variations')
    valid_recettes, valid_indexes  = data_analysis(recipe_dict, ref=False, col_name='Valid_Variations')
    test_recettes, test_indexes  = data_analysis(recipe_dict, ref=False, col_name='Test_Variations')
    
    recette_variations = train_recettes + valid_recettes + test_recettes
    KB_matrix,  KB_dist, Count_matrix = docs_distribution(KB_recettes, recette_variations)
    KB_size = list(range(KB_matrix.shape[0]))
    
    ## We set to 0 the distance here for each recipe -- difference needs to estimate distance between all points. 
    #This serves as optim to not calculate for each varaitions but only once since it is the same distance for all KB
    print('train variations for recipe {} done'.format(recette), ' | train variation size : ', len(train_recettes))
    neighboroud_distance  = 0. 
    for i in tqdm(range(len(train_recettes))):
        select_variation = KB_size + [len(KB_size)+i]
        NewKB_dist, variation_dist = new_distribution(Count_matrix, select_variation)
        
        KB_updated = KB_recettes + [train_recettes[i]]
        updated_text = ' '.join(KB_updated).split()
        New_EB_PMI = pmi(updated_text)
        
        #return newness, novelty_new, uniqueness, novelty_uniq, diff_ratio, nolvety_diff, neighbor_dist, newratio_surprise_rate, newn_suprise, dist_surprise, uniq_surprise
        newness, novelty_new, uniqueness, novelty_uniq, difference, \
        nolvety_diff, neighboroud_distance, new_surprise, novelty_nsuprise, \
        dist_surprise, novelty_dsurprise = compute_scores(KB_matrix, KB_dist, NewKB_dist, variation_dist, EB_PMI, dict_know_pmi, New_EB_PMI, neighbor_dist=neighboroud_distance)

        current_index = train_indexes[i]
        recipe_dict['Train_Variations'][current_index]['newness'] = newness
        recipe_dict['Train_Variations'][current_index]['novelty_new'] = novelty_new
        recipe_dict['Train_Variations'][current_index]['uniqueness'] = uniqueness
        recipe_dict['Train_Variations'][current_index]['novelty_uniq'] = novelty_uniq
        recipe_dict['Train_Variations'][current_index]['difference'] = difference
        recipe_dict['Train_Variations'][current_index]['nolvety_diff'] = nolvety_diff
        recipe_dict['Train_Variations'][current_index]['new_surprise'] = new_surprise
        recipe_dict['Train_Variations'][current_index]['novelty_nsuprise'] = novelty_nsuprise
        recipe_dict['Train_Variations'][current_index]['dist_surprise'] = dist_surprise
        recipe_dict['Train_Variations'][current_index]['novelty_dsurprise'] = novelty_dsurprise
    
    print('valid variations for recipe {} done'.format(recette), ' | valid variation size : ', len(valid_recettes))
    for i in tqdm(range(len(valid_recettes))):

        select_variation = KB_size + [(len(KB_size)+len(train_recettes))+i]
        NewKB_dist, variation_dist = new_distribution(Count_matrix, select_variation)

        KB_updated = KB_recettes + [valid_recettes[i]]
        updated_text = ' '.join(KB_updated).split()
        New_EB_PMI = pmi(updated_text)
        
        newness, novelty_new, uniqueness, novelty_uniq, diff_ratio, \
        nolvety_diff, neighboroud_distance, newratio_surprise_rate, newn_suprise, \
        dist_surprise, uniq_surprise = compute_scores(KB_matrix, KB_dist, NewKB_dist, variation_dist, EB_PMI, dict_know_pmi, New_EB_PMI, neighbor_dist=neighboroud_distance)
        current_index = valid_indexes[i]
        recipe_dict['Valid_Variations'][current_index]['newness'] = newness
        recipe_dict['Valid_Variations'][current_index]['novelty_new'] = novelty_new
        recipe_dict['Valid_Variations'][current_index]['uniqueness'] = uniqueness
        recipe_dict['Valid_Variations'][current_index]['novelty_uniq'] = novelty_uniq
        recipe_dict['Valid_Variations'][current_index]['difference'] = difference
        recipe_dict['Valid_Variations'][current_index]['nolvety_diff'] = nolvety_diff
        recipe_dict['Valid_Variations'][current_index]['new_surprise'] = new_surprise
        recipe_dict['Valid_Variations'][current_index]['novelty_nsuprise'] = novelty_nsuprise
        recipe_dict['Valid_Variations'][current_index]['dist_surprise'] = dist_surprise
        recipe_dict['Valid_Variations'][current_index]['novelty_dsurprise'] = novelty_dsurprise
    
    print('test variations for recipe {} done'.format(recette), ' | test variation size : ', len(test_recettes))
    for i in tqdm(range(len(test_recettes))):

        select_variation = KB_size + [(len(KB_size)+len(train_recettes)+len(valid_recettes))+i]
        NewKB_dist, variation_dist = new_distribution(Count_matrix, select_variation)

        KB_updated = KB_recettes + [test_recettes[i]]
        updated_text = ' '.join(KB_updated).split()
        New_EB_PMI = pmi(updated_text)
        
        newness, novelty_new, uniqueness, novelty_uniq, diff_ratio, \
        nolvety_diff, neighboroud_distance, newratio_surprise_rate, newn_suprise, \
        dist_surprise, uniq_surprise = compute_scores(KB_matrix, KB_dist, NewKB_dist, variation_dist, EB_PMI, dict_know_pmi, New_EB_PMI, neighbor_dist=neighboroud_distance)
        current_index = test_indexes[i]
        recipe_dict['Test_Variations'][current_index]['newness'] = newness
        recipe_dict['Test_Variations'][current_index]['novelty_new'] = novelty_new
        recipe_dict['Test_Variations'][current_index]['uniqueness'] = uniqueness
        recipe_dict['Test_Variations'][current_index]['novelty_uniq'] = novelty_uniq
        recipe_dict['Test_Variations'][current_index]['difference'] = difference
        recipe_dict['Test_Variations'][current_index]['nolvety_diff'] = nolvety_diff
        recipe_dict['Test_Variations'][current_index]['new_surprise'] = new_surprise
        recipe_dict['Test_Variations'][current_index]['novelty_nsuprise'] = novelty_nsuprise
        recipe_dict['Test_Variations'][current_index]['dist_surprise'] = dist_surprise
        recipe_dict['Test_Variations'][current_index]['novelty_dsurprise'] = novelty_dsurprise
    
    file_name = save_path + recette
    with open(file_name, "w") as outfile:
        json.dump(recipe_dict, outfile)

    print('file {} saved : '.format(k), recette)
    # Reset the start time to measure time per iteration
    start_time = time.time()
    del recipe_dict

Knowledge base size :  34
train variations for recipe Marinade_recipe_2.json done  | train variation size :  211


100%|████████████████████████████████████████████████████████████████████████████████| 211/211 [02:19<00:00,  1.51it/s]


valid variations for recipe Marinade_recipe_2.json done  | valid variation size :  34


100%|██████████████████████████████████████████████████████████████████████████████████| 34/34 [00:21<00:00,  1.56it/s]


test variations for recipe Marinade_recipe_2.json done  | test variation size :  42


...

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.16it/s]


test variations for recipe Chicken Satay_recipe_3.json done  | test variation size :  9


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:07<00:00,  1.19it/s]


file 4 saved :  Chicken Satay_recipe_3.json


JSONDecodeError: Expecting property name enclosed in double quotes: line 29 column 9 (char 2407)